In [1]:
!pip install neurokit2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.2/689.2 kB 15.4 MB/s eta 0:00:0000:01


In [2]:
!pip install hrv-analysis
!pip install pyhrv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.8/230.8 kB 5.8 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 57.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.0 MB/s eta 0:00:00
  Created wheel for spectrum: filename=spectrum-0.8.1-cp310-cp310-linux_x86_64.whl size=227335 sha256=813cfcbe48e92f63244f8bbac0033cd3bf037cd7db20f7eeda941645397d2374
  Stored in directory: /root/.cache/pip/wheels/e7/5a/09/ffc6afdf8a5a6f58e9851292108df32bb11374e11b8705cabd
Successfully built spectrum
  Attempting uninstall: pexpect
    Found existing installation: pexpect 4.8.0
    Uninstalling pexpect-4.8.0:
      Successfully uninstalled pexpect-4.8.0


In [3]:
#import tensorflow as tf
#import seaborn as sns

In [4]:
#from scipy.interpolate import interp1d

In [3]:
import numpy as np 
import pandas as pd 

In [5]:

from joblib import load
import keras
import neurokit2 as nk
import pyhrv.tools as tools
import pyhrv.frequency_domain as fd
import matplotlib
pd.options.mode.chained_assignment = None
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

2024-06-19 16:13:48.140494: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-19 16:13:48.140633: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-19 16:13:48.318254: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
from scipy import signal

In [7]:
import scipy 
import nltk
import sklearn
import keras
import pyhrv
import matplotlib
print('The nltk version is {}.'.format(nltk.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))
print('The scipy version is {}.'.format(scipy.__version__))
print('The keras version is {}.'.format(keras.__version__))
print('The pyhrv version is {}.'.format(pyhrv.__version__))
print('The matplotlib version is {}.'.format(matplotlib.__version__))

The nltk version is 3.2.4.
The scikit-learn version is 1.2.2.
The scipy version is 1.11.4.
The keras version is 3.3.3.
The pyhrv version is 0.4.1.
The matplotlib version is 3.7.5.


In [8]:
!pip install --upgrade scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 68.0 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


In [10]:
class ECG_RR():
    def __init__(self,info,ecg_signals_1,name):
        self.ECG = info
        self.ecg_signals_1 = ecg_signals_1
        self.name = name
    def RR(self,sp,RR_d):
        RR_d = RR_d
        if len(self.ECG[self.name]) > 3:
            for i in range(0,len(self.ECG[self.name])-1):
                RR_d = sp(RR_d,list(self.ECG[self.name])[i+1] - list(self.ECG[self.name])[i])
        else :RR_d =0
        return RR_d
    def R_m(self,sp):
        if len(self.ecg_signals_1[self.ecg_signals_1[self.name]>0].ECG_Raw):
            return sp(self.ecg_signals_1[self.ecg_signals_1[self.name]>0].ECG_Raw)
        else: return 0

In [11]:
def timedomain_pic(ecg_signals_1,info,name,results):
    ECG = ECG_RR(info,ecg_signals_1,"ECG_R_Peaks")
    name = str(name)+"_"
    results[name+'Max_R'] = [ECG.R_m(max)]
    results[name+'Min_R'] = [ECG.R_m(min)]
    results[name+'Max_RR'] = [ECG.RR(max,0)*(1/500)]
    results[name+'Min_RR'] = [ECG.RR(min,500)*(1/500)]
    ECG = ECG_RR(info,ecg_signals_1,"ECG_P_Peaks")
    results[name+'Max_P'] = [ECG.R_m(max)]
    results[name+'Min_P'] = [ECG.R_m(min)]
    ECG = ECG_RR(info,ecg_signals_1,"ECG_Q_Peaks")
    results[name+'Max_Q'] = [ECG.R_m(max)]
    results[name+'Min_Q'] = [ECG.R_m(min)]
    ECG = ECG_RR(info,ecg_signals_1,"ECG_S_Peaks")
    results[name+'Max_S'] = [ECG.R_m(max)]
    results[name+'Min_S'] = [ECG.R_m(min)]
    ECG = ECG_RR(info,ecg_signals_1,"ECG_T_Peaks")
    results[name+'Max_T'] = [ECG.R_m(max)]
    results[name+'Min_T'] = [ECG.R_m(min)]
    return results

In [12]:
def timedomain_HRV(ecg_signals_1,info,name,results):
    try:
        rr_intervals_list = info['ECG_R_Peaks']
        nni_2 = tools.nn_intervals(rr_intervals_list)
        result = fd.welch_psd(nni=nni_2,show=False, show_param=False,legend=False)
        matplotlib.pyplot.close(fig="all")
    except:
        return results
    name = str(name)+"_"
    results[name+'VLF_Peak (Hz)'] = [result[1][0]]
    results[name+'LF_Peak (Hz)'] = [result[1][1]]
    results[name+'HV_Peak (Hz)'] = [result[1][2]]
    results[name+'VLF_Abs (ms2)'] = [result[2][0]]
    results[name+'LF_Abs (ms2)'] = [result[2][1]]
    results[name+'HV_Abs (ms2)'] = [result[2][2]]
    results[name+'VLF_Rel (%)'] = [result[3][0]]
    results[name+'LF_Rel (%)'] = [result[3][1]]
    results[name+'HV_Rel (%)'] = [result[3][2]]
    results[name+'VLF_Log (-)'] = [result[4][0]]
    results[name+'LF_Log (-)'] = [result[4][1]]
    results[name+'HV_Log (-)'] = [result[4][2]]
    results[name+'LF_Norm (-)'] = [result[5][0]]
    results[name+'HV_Norm (-)'] = [result[5][1]]
    results[name+'LF/HF (-)'] = [result[6]]
    results[name+'Total Power (ms)'] = [result[7]]
    return results

In [13]:
def timedomain(ecg_signals_1,info,name,results):
    rr = info['ECG_R_Peaks'][:-1]
    name = str(name)+"_"
    rr_ecg = np.diff(rr)
    rr = rr_ecg*10
    hr = 60000/rr
    results[name+'Mean RR (ms)'] = [np.mean(rr)]
    results[name+'STD RR/SDNN (ms)'] = [np.std(rr)]
    results[name+'Mean HR (Kubios\' style) (beats/min)'] = [60000/np.mean(rr)]
    results[name+'Mean HR (beats/min)'] = [np.mean(hr)]
    results[name+'STD HR (beats/min)'] = [np.std(hr)]
    results[name+'Min HR (beats/min)'] = [np.min(hr)]
    results[name+'Max HR (beats/min)'] = [np.max(hr)]
    results[name+'RMSSD (ms)'] = [np.sqrt(np.mean(np.square(np.diff(rr))))]
    results[name+'NNxx'] = [np.sum(np.abs(np.diff(rr)) > 50)*1]
    results[name+'pNNxx (%)'] = [100 * np.sum((np.abs(np.diff(rr)) > 50)*1) / len(rr)]
    return results

In [14]:
def ecg_HRV_df(ecg_signals_1,info,name):
    results = {}
    results = pd.DataFrame(results)
    results =timedomain(ecg_signals_1,info,name,results)
    results =timedomain_HRV(ecg_signals_1,info,name,results)
    results =timedomain_pic(ecg_signals_1,info,name,results)
    return results

In [15]:
def ecg_HRV(ecg_signal,n_0,n_1):
    df_HTV =pd.DataFrame()
    #df = pd.DataFrame(ecg_signal[0])
    df = ecg_signal
    if n_1 - n_0 != 1:
        for j in range(n_0,n_1):

            ecg_signals_1, info  = nk.ecg_process(df[j], sampling_rate=500)
            HTV = ecg_HRV_df(ecg_signals_1,info,j)
            df_HTV[HTV.columns] = HTV
    elif n_1 - n_0 == 1:
        ecg_signals_1, info  = nk.ecg_process(df, sampling_rate=500)
        HTV = ecg_HRV_df(ecg_signals_1,info,n_0)
        df_HTV[HTV.columns] = HTV
    return df_HTV

In [16]:
def stardant_N(X_columns,fix_displacement_min, fix_displacement):
    for i in fix_displacement_min.columns:   
        X_columns[i] = X_columns[i] - fix_displacement_min[i]["min"]
    for i in fix_displacement.columns:   
        X_columns[i] = (X_columns[i] - fix_displacement[i]["mean"])/(fix_displacement[i]["std"])
    return X_columns 

## 2 


In [17]:
name_model = {
    "r":["LGBMClassifier.joblib","LinearSVC.joblib","lstm_model_r.keras"],
    "d":["LGBMClassifier.joblib","ExtraTreesClassifier.joblib","HistGradientBoostingClassifier.joblib","lstm_model_d.keras"]
}



In [18]:
def path_s(led,name_led,name_model,name_d,lan,HRV,path_def) :
    if led != "12_led/":
        path_n = led + name_model.split(".")[0]
        path_end = str(name_led-int(led.split("_")[0]))+"_"+str(name_led)+"_led_"+name_d + "."+name_model.split(".")[1]
    else :
        path_n = led + name_model.split(".")[0]
        path_end = "_"+str(name_led)+"_led_"+name_d + "."+name_model.split(".")[1]

        
    if name_d == "d":
        uniques = pd.read_csv( path_def +"diagnostic_2/uniques_diagnostic.csv", index_col=0)
        list_corr = pd.read_csv(path_def+"diagnostic_2/list_corr_diagnostic.csv", index_col=0)
        path = path_def + "diagnostic_2/" + path_n+path_end
        path_2 = path_def + "diagnostic_2/"+ path_n + "."+name_model.split(".")[1]
    if name_d == "r":
        uniques = pd.read_csv( path_def +"rhythm_2/uniques_rhythm.csv", index_col=0)
        list_corr = pd.read_csv(path_def+"rhythm_2/list_corr_rhythm.csv", index_col=0)
        path = path_def + "rhythm_2/" + path_n+path_end
        path_2 = path_def + "rhythm_2/"+ path_n + "."+name_model.split(".")[1]
    if "."+name_model.split(".")[1] == ".joblib":
            model = load(path)
            y_predict = model.predict(HRV)
            return uniques[lan][y_predict[0]]
    elif ("."+name_model.split(".")[1] == ".keras")and led == "12_led/":
        model = keras.models.load_model(path_2)
        y_predict = model.predict(HRV[list(list_corr["0"])[:-1]])
        max_in_ = pd.DataFrame(y_predict[0]).idxmax()[0]
        #pred_namber = pd.DataFrame(y_predict[0])*100
        #pred_namber[lan] = uniques[lan]
        return uniques[lan][max_in_]#,#pred_namber

In [19]:
class func_ecg_detect():
    def __init__(self,ECG,age,sex):
        self.path_def = "/kaggle/input/new-hrw-d/"
        self.ECG = ECG    
        self.age = age
        self.sex = sex
        self.fix_displacement = pd.read_csv(str(self.path_def + "fix_displacement.csv"), index_col=0)
        self.fix_displacement_min = pd.read_csv(str(self.path_def + "fix_displacement_min.csv"), index_col=0)

        
    def detect_led(self,count_led,name_led,name_model_1,name_d,lan):

        self.HRV_2 = ecg_HRV(self.ECG,name_led-count_led,name_led)
        self.HRV_2["age"] = self.age
        self.HRV_2["sex"] = self.sex
        self.HRV_2 = stardant_N(self.HRV_2,self.fix_displacement_min[self.HRV_2.columns], self.fix_displacement[self.HRV_2.columns])

        for i in name_model[name_diagnostic] :
            if name_model_1 in i:
                model = i
        path_2 = path_s(str(count_led)+"_led/",name_led,model,name_d,lan,self.HRV_2,self.path_def)

        return path_2
    
    

Проверка

In [ ]:
import os
!pip install wget
import wget
import wfdb

# Загрузка сигнала

In [4]:
d_measurements = pd.read_csv("/kaggle/input/electrocardiogram-database-for-arrhythmia-study/d_measurements.csv", index_col=0)

In [ ]:
def load_signals_mimic(df):
    data = [wfdb.rdsamp(str(df.name_file[link]), pn_dir='mimic-iv-ecg/1.0/'+ df.path[link] + "/") for link in df.index]
    data = np.array([signal for signal, meta in data])
    return data

In [5]:
d_measurements[5000:5001]

,report,report_1,study_id,report_0,path,name_file
11305210,LVH,left ventricular hypertrophy,48626808.0,sinus rhythm,files/p1130/p11305210/s48626808,48626808


In [ ]:
ecg_signal = load_signals_mimic(d_measurements[5000:5001])

In [ ]:
name_model[name_diagnostic][-1]

In [56]:
Age, Sex = 59,1
# count_led количиство отведений
count_led = 12 
# name_otvedenie от 1 до 12 при диагностике 1 отведения
# 6 - при диагностике отведения первых 6 отведений,12 - при диагностике с 6 по 12 (грудные) 
# 12 -  при диагностике всех
name_otvedenie = 12
# диагноз - d, ритм - r
name_diagnostic = "d" 
name_model_1 = "HistGradientBoostingClassifier.joblib"#name_model[name_diagnostic][1]
ecg_n_1 = pd.DataFrame(ecg_signal[0])

h = func_ecg_detect(ecg_n_1,Age, Sex)
h.detect_led(count_led,name_otvedenie,name_model_1,name_diagnostic,"ru")

'Гипертрофия левого желудочка'

In [48]:
name_diagnostic = "r"
name_model_1 = name_model[name_diagnostic][-1]
h = func_ecg_detect(ecg_n_1,Age, Sex)
h.detect_led(count_led,name_otvedenie,name_model_1,name_diagnostic,"ru")

/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 16 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step


'Синусовый ритм'

In [52]:
Age, Sex = 59,1
count_led = 6 
name_otvedenie = 6
ecg_n_1 = pd.DataFrame(ecg_signal[0])[[0,1,2,3,4,5]]

name_diagnostic = "r" 
name_model_1 = name_model[name_diagnostic][1]

h = func_ecg_detect(ecg_n_1,Age, Sex)
h.detect_led(count_led,name_otvedenie,name_model_1,name_diagnostic,"ru")

'Синусовый ритм'

In [53]:
name_otvedenie = 12
ecg_n_1 = pd.DataFrame(ecg_signal[0])[[6,7,8,9,10,11]]

name_diagnostic = "r" 
name_model_1 = name_model[name_diagnostic][1]

h = func_ecg_detect(ecg_n_1,Age, Sex)
h.detect_led(count_led,name_otvedenie,name_model_1,name_diagnostic,"ru")

'Синусовый ритм'

In [55]:
Age, Sex = 59,1
count_led = 1 
name_otvedenie = 4
ecg_n_1 = pd.DataFrame(ecg_signal[0])[4-1] # нумерация с 0 у индексов

name_diagnostic = "d" 
name_model_1 = name_model[name_diagnostic][1]

h = func_ecg_detect(ecg_n_1,Age, Sex)
h.detect_led(count_led,name_otvedenie,name_model_1,name_diagnostic,"ru")

'Гипертрофия левого желудочка'